<a href="https://colab.research.google.com/github/ThassiAmorim/Simulador-corrida/blob/main/Pdf2Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install PyMuPDF openpyxl pytesseract pillow
! apt install libtesseract-dev
!sudo apt install tesseract-ocr
!sudo apt-get install tesseract-ocr-por
import fitz  # PyMuPDF
import openpyxl
import pytesseract
from PIL import Image
import os
import cv2


# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho para o arquivo PDF
pdf_path = '/content/drive/MyDrive/AutomatizaNotas/notas-1-5.pdf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 106.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 3,743 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Fetched 3,743 kB in 1s (3,991 kB/s)
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 120831 files and directories cu

In [ ]:
from difflib import SequenceMatcher

def calculate_similarity(str1, str2):
    # Inicializar o objeto SequenceMatcher com as duas strings
    sequence_matcher = SequenceMatcher(None, str1, str2)

    # Calcular a similaridade como uma proporção entre as duas strings
    similarity_ratio = sequence_matcher.ratio()

    # Mapear a similaridade para uma escala de 0 a 10
    similarity_score = round(similarity_ratio * 10, 2)

    return similarity_score

# Função para extrair texto de uma imagem usando OCR
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    print(text)
    return text

# Função para extrair e converter imagens em texto
def extract_images_and_convert_to_text(page):
    text = ""
    images = page.get_images(full=True)

    for img_index, img in enumerate(images):
        xref = img[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]

        # Salvar a imagem em um arquivo temporário para que o pytesseract possa lê-la
        image_path = f"temp_image_{img_index}.png"
        with open(image_path, "wb") as image_file:
            image_file.write(image_bytes)

        # Usar pytesseract para converter a imagem em texto
        extracted_text = pytesseract.image_to_string(image_path, lang='por')

        # Adicionar o texto extraído à saída
        text += f"Imagem {img_index + 1} (Página {page_num + 1}):\n{extracted_text}\n\n"


        # Remover o arquivo de imagem temporário
        image_file.close()

    return text

# Função para extrair dados do texto
def extract_data_from_text(text):
    lines = text.split('\n')
    escola = ""
    total = ""

    for line in lines:
        print(f"-------------{line}---------------------")
        if "Estabelecimento:" in line:
            escola = line.split("Estabelecimento:")[1].strip()
            partes = escola.split("EF", 1)
            escola = partes[0]
        if calculate_similarity("VALOR TOTAL DA GUIA DE ENTREGA R$ ", line) > 8:
            try:
              total = line.split("R$ ")[1].strip()
            except:
              total = line.split("RS ")[1].strip()
            if total[len(total)-1] == ']' or total[len(total)-1] == ')':
              total = total[:-1]
            #return ...

    print(f"-----------{escola} ------------- {total}")

    return escola, total

# Abrir o PDF com PyMuPDF
pdf_document = fitz.open(pdf_path)

# Criar um novo arquivo Excel
excel_path = '/content/drive/MyDrive/AutomatizaNotas/extractions.xlsx'
workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title = "Extractions"
worksheet.append(["escola", "total"])

# Extrair dados das imagens e adicionar à planilha Excel
for page_num in range(pdf_document.page_count):
    page = pdf_document[page_num]

    escola, total = extract_data_from_text(extract_images_and_convert_to_text(page))
    worksheet.append([escola, total])

# Salvar o arquivo Excel
workbook.save(excel_path)

# Fechar o documento PDF
#pdf_document.close()

print("Extração e criação do Excel concluídas!")

-------------Imagem 1 (Página 1):---------------------
------------- ---------------------
----------------------------------
-------------COOPERATIVA DA AGRICULTURA FAMILIAR INTEGRADA DE CAPANEMA — COOPAFI CAPANEMA---------------------
----------------------------------
------------- ---------------------
----------------------------------
-------------Guia de Entrega de Alimentos da Agricultura Familiar - PNAE PR---------------------
----------------------------------
------------- ---------------------
----------------------------------
------------- ---------------------
----------------------------------
------------- ---------------------
----------------------------------
-------------Zhamada Pública Nº Contrato: 83/2022 | Data Emissão |Guia---------------------
-------------Edit al Nº 001/2021 04/04/2023 |Nº: 564---------------------
-------------Ago de Referência. 2023 Meio: Ma---------------------
-------------Município Estabelecimunto: Dep. Adm Zona:---------------------
---